<a href="https://colab.research.google.com/github/adityapribadi3/Privacy-Machine-Learning/blob/main/sgx_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install syft==0.2.9

     |████████████████████████████████| 433 kB 8.8 MB/s 
     |████████████████████████████████| 1.8 MB 45.7 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 2.0 MB 50.2 MB/s 
     |████████████████████████████████| 126 kB 70.9 MB/s 
     |████████████████████████████████| 66 kB 4.1 MB/s 
     |████████████████████████████████| 484 kB 40.0 MB/s 
     |████████████████████████████████| 200 kB 43.5 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 73 kB 1.9 MB/s 
     |████████████████████████████████| 4.0 MB 41.9 MB/s 
     |████████████████████████████████| 449 kB 50.3 MB/s 
     |████████████████████████████████| 9.0 MB 32.3 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 753.4 MB 4.7 kB/s 
     |████████████████████████████████| 3.2 MB 36.5 MB/s 
     |██████████████████

In [ ]:
import torch
import syft as sy
from torch import nn, optim

hook = sy.TorchHook(torch)

In [ ]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [ ]:
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [ ]:
model = nn.Linear(2,1)

bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [ ]:
for a_iter in range(10):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(5):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

preds = model(data)
loss = ((preds - target) ** 2).sum()

print(preds)
print(target)
print(loss.data)

Bob:tensor(0.0687) Alice:tensor(0.0035)
Bob:tensor(0.0467) Alice:tensor(0.0004)
Bob:tensor(0.0327) Alice:tensor(4.2396e-05)
Bob:tensor(0.0234) Alice:tensor(5.0751e-07)
Bob:tensor(0.0170) Alice:tensor(2.2041e-05)
Bob:tensor(0.0125) Alice:tensor(4.5905e-05)
Bob:tensor(0.0093) Alice:tensor(5.9106e-05)
Bob:tensor(0.0070) Alice:tensor(6.2270e-05)
Bob:tensor(0.0053) Alice:tensor(5.8928e-05)
Bob:tensor(0.0040) Alice:tensor(5.2272e-05)
tensor([[0.1742],
        [0.1403],
        [0.8237],
        [0.7898]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.1253)
